In [1]:
from backend.src.input_processing.date_data_extraction import DataExtractor
f = DataExtractor()

d =DataExtractor()
k =d.get_transformed_data_v1("08/04/2025, 03:34","san Jose","santa clara","95377")

k






(1, 78)


array([[ 1.44020859, -2.80572995,  0.50242963, -0.75142855,  0.96899711,
        -0.76580748,  1.15973417,  0.23099606,  0.05312002, -0.05594495,
        -0.44412661, -0.54467354,  0.10762743, -0.00335709,  0.03096319,
        -0.09785111, -0.01068048,  0.02662238, -0.16336684,  0.17297668,
        -0.31830198,  0.17944857, -0.08162294, -0.10676247,  0.59478867,
        -0.40614046,  0.02673223]])

In [2]:
import pandas as pd
from torch.cuda import temperature

raw_data = pd.read_csv("../data/CA_Accidents.csv")

raw_data.shape

KeyboardInterrupt: 

In [ ]:
data = {'request': {'type': 'City', 'query': 'San Jose, United States of America', 'language': 'en', 'unit': 'f'}, 'location': {'name': 'San Jose', 'country': 'United States of America', 'region': 'California', 'lat': '37.339', 'lon': '-121.894', 'timezone_id': 'America/Los_Angeles', 'localtime': '2025-08-04 03:48', 'localtime_epoch': 1754279280, 'utc_offset': '-7.0'}, 'current': {'observation_time': '10:48 AM', 'temperature': 61, 'weather_code': 143, 'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'], 'weather_descriptions': ['Mist'], 'astro': {'sunrise': '06:15 AM', 'sunset': '08:12 PM', 'moonrise': '05:11 PM', 'moonset': '01:25 AM', 'moon_phase': 'Waxing Gibbous', 'moon_illumination': 73}, 'air_quality': {'co': '288.6', 'no2': '6.29', 'o3': '55', 'so2': '3.145', 'pm2_5': '13.69', 'pm10': '18.685', 'us-epa-index': '1', 'gb-defra-index': '1'}, 'wind_speed': 2, 'wind_degree': 230, 'wind_dir': 'SW', 'pressure': 1017, 'precip': 0, 'humidity': 67, 'cloudcover': 0, 'feelslike': 61, 'uv_index': 0, 'visibility': 10, 'is_day': 'no'}}

# Extracted from the "current" section
current = data["current"]

# Weather features
temperature_f = current["temperature"]
humidity_percent = current["humidity"]
pressure_in = round(current["pressure"] * 0.02953, 2)        # hPa to inHg
visibility_mi = current["visibility"]
wind_speed_mph = current["wind_speed"]
wind_chill_f = current["feelslike"]
precipitation_in = round(current["precip"] * 0.03937, 2)      # mm to inches


weather_data = {
    "Temperature(F)": temperature_f,
    "Humidity(%)": humidity_percent,
    "Pressure(in)": pressure_in,
    "Visibility(mi)": visibility_mi,
    "Wind_Speed(mph)": wind_speed_mph,
    "Wind_Chill(F)": wind_chill_f,
    "Precipitation(in)": precipitation_in
}

print(weather_data)



In [ ]:
# Library holidays allows us flag as holiday or not.
import holidays
import pandas as pd

class DataExtractor:
    def __init__(self):
        self.us_holidays = holidays.US()
        pass

    def get_date_data(self, data):


        # Step 1: Convert the string to datetime
        dt = pd.to_datetime(data.strip(), format="%m/%d/%Y, %H:%M")

        # Step 2: Extract components
        hour = dt.hour
        day_of_week = dt.day_name()
        month = dt.month_name()

        # Step 3: Check if it's a holiday (US by default, or specify country)
        us_holidays = holidays.US()
        is_holiday = dt.date() in us_holidays

        # Step 4: Define rush hour (e.g., 6–9 AM and 4–7 PM)
        is_rush_hour = (6 <= hour <= 9) or (16 <= hour <= 19)

        date_data = {"day_of_week":day_of_week,"hour":hour,"month":month,"is_holiday":is_holiday,"is_rush_hour":is_rush_hour}

        return date_data

    def get_weather_data(self):
        import requests

        api_key = "e8596ef49660d3eebea854a77bca7faa"
        location = "San Jose"  # You can use city or "City, Country"

        url = f"http://api.weatherstack.com/current?access_key={api_key}&query={location}&units=f"

        response = requests.get(url)
        data = response.json()

        current = data["current"]

        # Weather features
        temperature_f = current["temperature"]
        humidity_percent = current["humidity"]
        pressure_in = round(current["pressure"] * 0.02953, 2)  # hPa to inHg
        visibility_mi = current["visibility"]
        wind_speed_mph = current["wind_speed"]
        wind_chill_f = current["feelslike"]
        precipitation_in = round(current["precip"] * 0.03937, 2)  # mm to inches

        weather_data = {
            "Temperature(F)": temperature_f,
            "Humidity(%)": humidity_percent,
            "Pressure(in)": pressure_in,
            "Visibility(mi)": visibility_mi,
            "Wind_Speed(mph)": wind_speed_mph,
            "Wind_Chill(F)": wind_chill_f,
            "Precipitation(in)": precipitation_in
        }
        return weather_data

    def get_weather_data_dev(self):


        data = {'request': {'type': 'City', 'query': 'San Jose, United States of America', 'language': 'en', 'unit': 'f'}, 'location': {'name': 'San Jose', 'country': 'United States of America', 'region': 'California', 'lat': '37.339', 'lon': '-121.894', 'timezone_id': 'America/Los_Angeles', 'localtime': '2025-08-04 03:48', 'localtime_epoch': 1754279280, 'utc_offset': '-7.0'}, 'current': {'observation_time': '10:48 AM', 'temperature': 61, 'weather_code': 143, 'weather_icons': ['https://cdn.worldweatheronline.com/images/wsymbols01_png_64/wsymbol_0006_mist.png'], 'weather_descriptions': ['Mist'], 'astro': {'sunrise': '06:15 AM', 'sunset': '08:12 PM', 'moonrise': '05:11 PM', 'moonset': '01:25 AM', 'moon_phase': 'Waxing Gibbous', 'moon_illumination': 73}, 'air_quality': {'co': '288.6', 'no2': '6.29', 'o3': '55', 'so2': '3.145', 'pm2_5': '13.69', 'pm10': '18.685', 'us-epa-index': '1', 'gb-defra-index': '1'}, 'wind_speed': 2, 'wind_degree': 230, 'wind_dir': 'SW', 'pressure': 1017, 'precip': 0, 'humidity': 67, 'cloudcover': 0, 'feelslike': 61, 'uv_index': 0, 'visibility': 10, 'is_day': 'no'}}

        current = data["current"]

        # Weather features
        temperature_f = current["temperature"]
        humidity_percent = current["humidity"]
        pressure_in = round(current["pressure"] * 0.02953, 2)  # hPa to inHg
        visibility_mi = current["visibility"]
        wind_speed_mph = current["wind_speed"]
        wind_chill_f = current["feelslike"]
        precipitation_in = round(current["precip"] * 0.03937, 2)  # mm to inches

        weather_data = {
            "Temperature(F)": temperature_f,
            "Humidity(%)": humidity_percent,
            "Pressure(in)": pressure_in,
            "Visibility(mi)": visibility_mi,
            "Wind_Speed(mph)": wind_speed_mph,
            "Wind_Chill(F)": wind_chill_f,
            "Precipitation(in)": precipitation_in
        }
        return weather_data


    def get_transformed_data(self, date_and_time, city, county,zipcode):

        date_data = self.get_date_data(date_and_time)
        weather_data = self.get_weather_data_dev()

        data = {
            'Temperature(F)': weather_data["Temperature(F)"],
            'Humidity(%)': weather_data["Humidity(%)"],
            'Pressure(in)': weather_data["Humidity(%)"],
            'Visibility(mi)':weather_data["Visibility(mi)"],
            'Wind_Speed(mph)':weather_data["Wind_Speed(mph)"],
            "Wind_Chill(F)": weather_data[],
            "Precipitation(in)": weather_data[]

        }


d =DataExtractor()
print(d.get_weather_data_dev())







